In [1]:
import matplotlib.pyplot as plt
import numpy as np

In [1]:
test = """
seeds: 79 14 55 13

seed-to-soil map:
50 98 2
52 50 48

soil-to-fertilizer map:
0 15 37
37 52 2
39 0 15

fertilizer-to-water map:
49 53 8
0 11 42
42 0 7
57 7 4

water-to-light map:
88 18 7
18 25 70

light-to-temperature map:
45 77 23
81 45 19
68 64 13

temperature-to-humidity map:
0 69 1
1 0 69

humidity-to-location map:
60 56 37
56 93 4
""".strip()

In [2]:
with open('input.txt', 'r') as f:
    input_ = f.read().strip()

# Part 1

In [95]:
class RangeMap:
    def __init__(self, dest_source_range_txt):
        dest_source_range = []
        for row in dest_source_range_txt.strip().splitlines():
            dest_start, source_start, range_len = [int(x) for x in row.split()]
            dest_source_range.append([source_start, source_start + range_len - 1, dest_start])
        # sorted by source_start
        self.dest_source_range = sorted(dest_source_range)

    def map(self, source):
        for source_start, source_end, dest_start in self.dest_source_range:
            if source < source_start:
                dest = source
                break
            if source >= source_start and source <= source_end:
                dest = source - source_start + dest_start
                break
        else:
            dest = source
        return dest

In [96]:
parts = test.split('\n\n')
seeds_txt = parts[0]
seeds = [int(s) for s in seeds_txt[7:].split()]

range_maps = []
for map_desc in parts[1:]:
    map_name, map_txt = map_desc.split(':')
    range_maps.append(RangeMap(map_txt))

destinations = []
for s in seeds:
    d = s
    for rm in range_maps:
        d = rm.map(d)
    destinations.append(d)
min(destinations)

35

In [97]:
parts = input_.split('\n\n')
seeds_txt = parts[0]
seeds = [int(s) for s in seeds_txt[7:].split()]

range_maps = []
for map_desc in parts[1:]:
    map_name, map_txt = map_desc.split(':')
    range_maps.append(RangeMap(map_txt))

destinations = []
for s in seeds:
    d = s
    for rm in range_maps:
        d = rm.map(d)
    destinations.append(d)
min(destinations)

165788812

# Part 2

In [256]:
class RangeMap:
    def __init__(self, dest_source_range_txt):
        dest_source_range = []
        for row in dest_source_range_txt.strip().splitlines():
            dest_start, source_start, range_len = [int(x) for x in row.split()]
            dest_source_range.append([source_start, source_start + range_len - 1, dest_start])
        # sorted by source_start
        dest_source_range = sorted(dest_source_range)
    
        # build a range map without holes
        pos = 0
        complete_dest_source_range = []
        for source_start, source_end, dest_start in dest_source_range:
            if pos < source_start:
                complete_dest_source_range.append((pos, source_start-1, pos))
            complete_dest_source_range.append((source_start, source_end, dest_start))
            pos = source_end + 1
        complete_dest_source_range.append((pos, 1_000_000_000_000, pos))
        self.dest_source_range = complete_dest_source_range
        
    def map_range(self, source_range_start, source_range_end):
        # Map an entire range in destination ranges
        mapped_range = []

        source = source_range_start
        end = False
        for source_start, source_end, dest_start in self.dest_source_range:
            if source > source_end:
                continue
            if source_range_end < source_end:
                source_end = source_range_end
                end = True
            dest = source - source_start + dest_start
            len_ = source_end - source + 1
            mapped_range.append((dest, dest+len_-1))
            source = source_end + 1
            if end:
                break

        return mapped_range


In [272]:
parts = test.split('\n\n')
seeds_txt = parts[0]
seeds_int = [int(s) for s in seeds_txt[7:].split()]

seeds_ranges = []
for i in range(0, len(seeds_int), 2):
    seeds_ranges.append((seeds_int[i], seeds_int[i+1]))
seeds_ranges

[(79, 14), (55, 13)]

In [273]:
range_maps = []
for map_desc in parts[1:]:
    map_name, map_txt = map_desc.split(':')
    range_maps.append(RangeMap(map_txt))

In [274]:
destinations = []
for seed_start, seed_len in sorted(seeds_ranges):
    source_ranges = [(seed_start, seed_start + seed_len - 1)]
    print('sr', source_ranges)
    for rm in range_maps:
        dest_ranges = []
        for start, end in source_ranges:
            mapped_range = rm.map_range(start, end)
            dest_ranges.extend(mapped_range)
        source_ranges = sorted(dest_ranges)
        print('d', dest_ranges)


sr [(55, 67)]
d [(57, 69)]
d [(57, 69)]
d [(53, 56), (61, 69)]
d [(46, 49), (54, 62)]
d [(82, 85), (90, 98)]
d [(82, 85), (90, 98)]
d [(86, 89), (94, 96), (56, 59), (97, 98)]
sr [(79, 92)]
d [(81, 94)]
d [(81, 94)]
d [(81, 94)]
d [(74, 87), (95, 94)]
d [(78, 80), (45, 55), (63, 62)]
d [(46, 56), (64, 63), (78, 80)]
d [(46, 55), (60, 60), (68, 67), (82, 84)]


In [275]:
source_ranges[0][0]

46

Now for the real input

In [276]:
parts = input_.split('\n\n')
seeds_txt = parts[0]
seeds_int = [int(s) for s in seeds_txt[7:].split()]

seeds_ranges = []
for i in range(0, len(seeds_int), 2):
    seeds_ranges.append((seeds_int[i], seeds_int[i+1]))
seeds_ranges

[(2494933545, 159314859),
 (4045092792, 172620202),
 (928898138, 554061882),
 (2740120981, 81327018),
 (2031777983, 63513119),
 (2871914181, 270575980),
 (2200250633, 216481794),
 (3289604059, 25147787),
 (3472625834, 10030240),
 (260990830, 232636388)]

In [277]:
range_maps = []
for map_desc in parts[1:]:
    map_name, map_txt = map_desc.split(':')
    range_maps.append(RangeMap(map_txt))


In [279]:
destinations = []
for seed_start, seed_len in sorted(seeds_ranges):
    source_ranges = [(seed_start, seed_start + seed_len - 1)]
    for rm in range_maps:
        dest_ranges = []
        for start, end in source_ranges:
            mapped_range = rm.map_range(start, end)
            dest_ranges.extend(mapped_range)
        source_ranges = sorted(dest_ranges)


In [280]:
source_ranges[0][0]

1928058